# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [320]:
%%time
import pandas as pd
import numpy as np
import datetime


def escrever_csv(dfa, nome): 
    dfa.to_csv('D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\%s.csv' %nome, index=False)

def escrever_txt(dfa, nome): 
    dfa.to_csv('D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\%s.txt' %nome, index=False, header=False)
    
def escrever_excel(dfa, nome):
    dfa.to_excel(f'D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\{nome}.xlsx' , index=False)

CPU times: total: 0 ns
Wall time: 0 ns


---

#  <span style="color:red"><u>Ler Ficheiro Completo</u> </span>

In [321]:
%%time

#Ler os ficheiros
df_2022 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2022\\Stocks_Delta_2022_Limpo.csv')
df_2023 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2023\\Stocks_Delta_2023_Limpo.csv')

# Juntar as bases
dataframes = [df_2022, df_2023]
df_Fusão = pd.concat(dataframes, ignore_index=True)
df_Fusão['DATA']= pd.to_datetime(df_Fusão['DATA'], format='%Y-%m-%d')
df_Fusão = df_Fusão.rename(columns={"INTRANSIT": "INTRANSIT_1_Dias_Antes", 
                                    "EXPECTED": "EXPECTED_1_Dias_Antes", 
                                    "PRES_STOCK": "PRES_STOCK_1_Dias_Antes"})


# Ficheiro de previsão
df_Prophet = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\Prophet.csv')
df_Prophet['DATA']= pd.to_datetime(df_Prophet['DATA'], format='%Y-%m-%d')

df_XGBoost = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\XGBoost.csv')
df_XGBoost['DATA']= pd.to_datetime(df_XGBoost['DATA'], format='%Y-%m-%d')

CPU times: total: 35.1 s
Wall time: 50.2 s


# <font color=red>Opcional:</font> Definir produtos em causa

- Produtos específicos

In [322]:
%%time
# Ler ficheiro dos produtos e lojas para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\lojas.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()
produtos+=["CAFÉ DELTA Q MYTHIQ 10CAP", "CAFÉ DELTA Q MYTHIQ XL 40CAP", "CAFÉ DELTA Q QALIDUS 10CAP", "CAFÉ DELTA Q QALIDUS 40CAP",
                'CAFÉ DELTA Q QHARACTER 10CAP','CAFÉ DELTA Q QHARACTER 40CAP','CAFÉ DELTA Q DEQAFEINATUS 10CAP','CAFÉ DELTA Q DEQAFEINATUS XL 40CAP',
               'CAFÉ DELTA MOAGEM UNIVERSAL ANGOLA 220G','CAFÉ DELTA MOAGEM UNIVERSAL BRASIL 220G']
# Alterar o dataframe para apenas incluir os produtos e lojas em causa
dfFinal = df_Fusão[(df_Fusão["DESC_ARTIGO"].isin(produtos)) & (df_Fusão["STORE_NAME"].isin(lojas))].copy()

CPU times: total: 1.03 s
Wall time: 1.53 s


In [323]:
df2 = pd.read_excel('D:\\B&N Dados\\Delta\\\Book3.xlsx')


num_repeats = len(dfFinal) // len(df2) + 1

repeated_values = np.tile(df2['FOTO'], num_repeats)[:len(dfFinal)]

dfFinal["FOTO"] = repeated_values

## <font color=red>Fim</font> 

In [324]:
dfFinal['INTRANSIT'] = dfFinal.groupby(["STORE","EAN"])['INTRANSIT_1_Dias_Antes'].shift(-1)
dfFinal['EXPECTED'] = dfFinal.groupby(["STORE","EAN"])['EXPECTED_1_Dias_Antes'].shift(-1)
dfFinal['PRES_STOCK'] = dfFinal.groupby(["STORE","EAN"])['PRES_STOCK_1_Dias_Antes'].shift(-1)
dfFinal["SELLOUT"] = np.where(dfFinal["SELLOUT"]<0, 0, dfFinal["SELLOUT"])
dfFinal["SELLOUT_1_Dias_Antes"] = np.where(dfFinal["SELLOUT_1_Dias_Antes"]<0, 0, dfFinal["SELLOUT_1_Dias_Antes"])

- Fundir o Ficheiro da Delta com a previsão

In [325]:
%%time
# Prophet
dfFinal = pd.merge(dfFinal, df_Prophet[['DATA', 'STORE', 'DESC_ARTIGO', 'Prophet']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )
# XGBoost
dfFinal = pd.merge(dfFinal, df_XGBoost[['DATA', 'STORE', 'DESC_ARTIGO', 'XGBoost']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )

CPU times: total: 625 ms
Wall time: 941 ms


- Vendedores

In [326]:
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfFinal = pd.merge(dfFinal, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA

> - PRÉ_ROTURA

In [327]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)
dfFinal["ROTURA_1_Dias_Antes"] = np.where((dfFinal["STOCK_1_Dias_Antes"] <= 0) & (dfFinal["PRES_STOCK_1_Dias_Antes"] > 0), 1, 0)


# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)
dfFinal["PRE_ROTURA_1_Dias_Antes"] = (dfFinal["STOCK_1_Dias_Antes"] < dfFinal["PRES_STOCK_1_Dias_Antes"]).astype(int)

# Métricas 1, 4, 5 e 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [329]:
# Quantos dias antes:

diasMet = [1, 4, 5, 10]

---

 ### <font color=green> Função Dias Antes

In [330]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,f'{coluna}_{a}_Dias_Antes'] = valores

> - STOCK
> - SELLOUT
> - INTRANSIT
> - EXPECTED 
> - STK
> - FORNECIMENTO
> - CICLOS
> - Adequação

In [331]:
dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]
dfFinal["FORNECIMENTO"] = dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]
dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]
dfFinal["Adequacao"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in diasMet:   
    dias(dfFinal, i, "STOCK")
    dias(dfFinal, i, "SELLOUT")
    dias(dfFinal, i, "INTRANSIT")
    dias(dfFinal, i, "EXPECTED")
    dias(dfFinal, i, "STK")
    dias(dfFinal, i, "FORNECIMENTO")
    dias(dfFinal, i, "CICLOS")
    dias(dfFinal, i, "Adequacao")

> - MSA

>- Balance: sellout / soma stock disponível mais transito.

In [332]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=10).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")
    

dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=20).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in diasMet:
    
    
    valores = i * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


---

> > - Ordenar

# Colunas de métricas 30, 60, 120 e 180 dias antes

> - Volatilidade de Procura: <br>
coeficiente de variação

In [334]:
diasMetHist = [30, 60, 120, 180]

### Percentagens

>- Balance raw

In [335]:
dfFinal["Balance_Raw"] =  dfFinal["SELLOUT"] / dfFinal["STK"]

dfFinal["Balance_Raw_Count1"] = np.where(dfFinal["Balance_Raw"] < 0.5, 1, 0)

dfFinal["Balance_Raw_Count2"] = np.where(dfFinal["Balance_Raw"] < 0.8, 1, 0)

for i in diasMetHist:
    dfFinal[f"Percentagem_Balance_Raw_Count1_{i}"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Balance_Raw_Count1'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())
    dfFinal[f"Percentagem_Balance_Raw_Count2_{i}"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Balance_Raw_Count2'].transform(lambda x: x.rolling(window=i, min_periods=1).mean())

> - Volatilidade

> - Rotura 

> - Supply

> - Percentagem de dias em Stock Borderline

> - Percentagem de dias de Linear Incompleto

> - Percentagem de dias sem vendas

> - Tempo indisponível

> - Vendas perdidas

In [337]:
# Sempre que é pedido abastecimento, fazer com que seja 1
dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED_1_Dias_Antes"]==0) & (dfFinal["EXPECTED"]>0), 1, 0)
dfFinal["Percentagem_Stock_Borderline_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<0.2*dfFinal["PRES_STOCK_1_Dias_Antes"], 1, 0)
dfFinal["Percentagem_Linear_Incompleto_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<dfFinal["PRES_STOCK_1_Dias_Antes"], 1, 0)
dfFinal["Sem_Vendas_1_Dias_Antes"] = np.where(dfFinal["SELLOUT_1_Dias_Antes"] == 0, 1, 0)
dfFinal["Dias_Indisponivel_1_Dias_Antes"] = np.where(dfFinal["ROTURA_1_Dias_Antes"]==1, 1, 0)

dfFinal['ROTURA_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["ROTURA_1_Dias_Antes"].copy()
dfFinal['ROTURA_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["ROTURA_1_Dias_Antes"].copy()

for i in diasMetHist:
    dfFinal[f"Percentagem_Volatilidade_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes']\
                                                .transform(lambda x: x.rolling(window=i, min_periods=1).std()) /
                                                dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes']\
                                                .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes']\
                                           .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes']\
                                           .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Supply_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply']\
                                          .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Percentagem_Dias_Stock_Borderline_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Stock_Borderline_1_Dias_Antes']\
                                                         .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Percentagem_Dias_Linear_Incompleto_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Linear_Incompleto_1_Dias_Antes']\
                                                          .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Dias_Sem_Vendas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas_1_Dias_Antes']\
                                                   .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))
    
    dfFinal[f"Percentagem_Dias_Indisponivel_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Dias_Indisponivel_1_Dias_Antes']\
                                                     .transform(lambda x: x.rolling(window=i, min_periods=1).mean()))

    dfFinal[f"Vendas_Perdidas_em_{i}_Dias"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum())\
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).median())) \
                                                  + 
                                               (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum()) \
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).median())))
    

> - Efeito fim de semana

In [339]:
#dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
#dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()
dfFinal['SELLOUT_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["SELLOUT_1_Dias_Antes"].copy()
dfFinal['SELLOUT_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["SELLOUT_1_Dias_Antes"].copy()

dfFinal['SELLOUT_fds_Medio'] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())
dfFinal['SELLOUT_semana_Medio'] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())


for i in diasMetHist:
    dfFinal[f"Percentagem_Efeito_Fds_{i}"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())/
                                              (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())))-1)


> - Tempo médio inter-supply

In [341]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

for i in diasMetHist:
    dfFinal[f"Percentagem_InterSupplyMed_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))


---

---

 ### <font color=green> Função Contagem

In [ ]:
%%time
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def contagem5dias(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})


# Define a function to calculate weekday and weekend counts
def contagem3dias(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(3):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA3": weekday_count, "CONTAGEM_FIMSEMANA3": weekend_count})

> - Smart 3 dias

In [353]:
# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA3", "CONTAGEM_FIMSEMANA3"]] = dfFinal["DATA"].apply(contagem3dias)
dfFinal['CONTAGEM_FIMSEMANA3'] = dfFinal['CONTAGEM_FIMSEMANA3'].shift(-1)
dfFinal['CONTAGEM_SEMANA3'] = dfFinal['CONTAGEM_SEMANA3'].shift(-1)

dfFinal["Balance_Smart3"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA3'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA3']))
     / dfFinal["STK"])

dfFinal["Balance_Smart3"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart3"])
dfFinal["Balance_Smart_3_Dias_Antes"] = dfFinal["Balance_Smart3"].shift(2)


CPU times: total: 25.9 s
Wall time: 26 s


> - Smart 5 dias antes

In [352]:
# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(contagem5dias)
dfFinal['CONTAGEM_FIMSEMANA'] = dfFinal['CONTAGEM_FIMSEMANA'].shift(-1)
dfFinal['CONTAGEM_SEMANA'] = dfFinal['CONTAGEM_SEMANA'].shift(-1)


dfFinal["Balance_Smart"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK"])

dfFinal["Balance_Smart"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart"])
dfFinal["Balance_Smart_5_Dias_Antes"] = dfFinal["Balance_Smart"].shift(4)


CPU times: total: 27 s
Wall time: 28.4 s


> - Dias para rotura de Stock

In [354]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STK_1_Dias_Antes"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_para_Rotura_Stock_5_Dias_Antes"] = dfFinal["Dias_para_Rotura_Stock"].shift(4)

> - Dias para rotura de Linear

In [355]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duracao_Linear'] = dfFinal["PRES_STOCK"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_Duracao_Linear_5_Dias_Antes"] = dfFinal["Dias_Duracao_Linear"].shift(4)

### Texto

> - Desempenho

> - Volatilidade de Sellout

> - Sensibilidade

> - PS Classificação

> - Risco de Rotura

In [356]:
dfFinal["Desempenho_de_Vendas_60"] = np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>=0) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.151), "Vendas com desempenho satisfatório",
                                     np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.151) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.317), "Vendas com desempenho mediano",
                                     np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.317) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=0.433), "Vendas com desempenho insatisfatório",
                                              "Vendas com desempenho muito insatisfatório")))


                                     #np.where((dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.433) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<=1), 

dfFinal["Volatilidade_SELLOUT"] = np.where(dfFinal["Percentagem_Volatilidade_60"] > 1.095, " e com alta volatilidade de sellout.",
                                  np.where(dfFinal["Percentagem_Volatilidade_60"] > 0.91 , " e com média volatilidade de sellout.",
                                  np.where(dfFinal["Percentagem_Volatilidade_60"] > 0.76 , " e com baixa volatilidade de sellout.",
                                                                                           " e com muito baixa volatilidade de sellout.")))


dfFinal["Sensibilidade_Rotura"] = np.where(dfFinal["Percentagem_Roturas_60"]>0.33, " Produto com muito elevada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.166666, " Produto com elevada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0.017, " Produto com moderada propensão para rotura",
                                 np.where(dfFinal["Percentagem_Roturas_60"]>0, " Produto com baixa propensão para rotura",
                                 " Produto com muito baixa ou nula propensão para rotura"))))

dfFinal["PS_Classificacao_60"] = np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.51) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==1) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS excessivo.",
                              np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] > 0.31) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]==1) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]>0.29), " e com PS moderadamente excessivo.",
                              np.where((dfFinal["Percentagem_Dias_Linear_Incompleto_60"] < 0.07) & (dfFinal["Percentagem_Balance_Raw_Count1_60"]<0.9) & (dfFinal["Percentagem_Dias_Sem_Vendas_60"]<0.07), " e com PS escasso.", 
                                                                                                                                                                                                            " e com PS equilibrado.")))

dfFinal["RISCO"] = np.where((dfFinal.Balance_Smart > 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.2) & (dfFinal.Percentagem_Roturas_120 > 5), 1,
                   np.where((dfFinal.Balance_Smart > 1) & (dfFinal.Balance_Smart < 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.6) & (dfFinal.Percentagem_Roturas_120 < 3), 2, 3))

dfFinal["RISCO_ROTURA"] = np.where((dfFinal.Balance_Smart >= 12.5) | (dfFinal.Balance_Smart < 0), " O produto apresenta hoje muito elevada probabilidade de rotura a 5 dias caso não haja fornecimento.", 
                          np.where((dfFinal.Balance_Smart >= 1.1) & (dfFinal.Balance_Smart < 12.5), " O produto apresenta hoje elevada probabilidade de rotura a 5 dias caso não haja fornecimento.",
                          np.where((dfFinal.Balance_Smart >= 0.833) & (dfFinal.Balance_Smart < 1.1), " O produto apresenta hoje média probabilidade de rotura a 5 dias caso não haja fornecimento.",
                          np.where((dfFinal.Balance_Smart >= 0.087) & (dfFinal.Balance_Smart < 0.833), " O produto apresenta hoje baixa probabilidade de rotura a 5 dias caso não haja fornecimento.", 
                                                                                                         " O produto apresenta hoje muito baixa ou nula probabilidade de rotura a 5 dias caso não haja fornecimento."))))         

> - Conclusão

In [363]:
dfFinal["Conclusao1"] = dfFinal["Desempenho_de_Vendas_60"] + dfFinal["Volatilidade_SELLOUT"] 
dfFinal["Conclusao2"] = dfFinal["Sensibilidade_Rotura"] + dfFinal["PS_Classificacao_60"] 
dfFinal["Conclusao3"] = dfFinal["RISCO_ROTURA"]

# Escrever

- Dias certos

In [364]:
#dfEscrever2022 = dfFinal.copy()
#dfEscrever2023 = dfFinal.loc[(dfFinal['DATA'] >= '2023-01-01') ].copy()

#df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
#df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

In [365]:
dfFinal["ROTURA"] = np.where(dfFinal["ROTURA"]==1, "SIM", "NÃO")
# Ficheiro Dia
dfEscreverDia = dfFinal[dfFinal.DATA == dfFinal.DATA.unique()[-2]].copy()

# Ficheiro Mês
dfEscreverMes = dfFinal[dfFinal.DATA.between(dfFinal.DATA.unique()[-31],dfFinal.DATA.unique()[-2])].copy()

# Ficheiro 180 dias
dfEscrever180 = dfFinal[dfFinal.DATA.between(dfFinal.DATA.unique()[-181],dfFinal.DATA.unique()[-2])].copy()

In [388]:
print(2022)
for i in range (1,13):
    
    print(f"Mês {i}: ",dfFinal[(dfFinal.ROTURA=="SIM") & (dfFinal.DATA.dt.year==2022) & (dfFinal.DATA.dt.month==i)].shape[0])
print(2023)  
for i in range (1,13):
    print(f"Mês {i}: ",dfFinal[(dfFinal.ROTURA=="SIM") & (dfFinal.DATA.dt.year==2023) & (dfFinal.DATA.dt.month==i)].shape[0])

2022
Mês 1:  50
Mês 2:  5
Mês 3:  175
Mês 4:  18
Mês 5:  32
Mês 6:  7
Mês 7:  7
Mês 8:  16
Mês 9:  195
Mês 10:  106
Mês 11:  6
Mês 12:  4
2023
Mês 1:  59
Mês 2:  11
Mês 3:  76
Mês 4:  12
Mês 5:  5
Mês 6:  0
Mês 7:  0
Mês 8:  0
Mês 9:  0
Mês 10:  0
Mês 11:  0
Mês 12:  0


In [395]:
dfFinal[(dfFinal.STORE==2) & (dfFinal.DATA.between(dfFinal.DATA.unique()[-61],dfFinal.DATA.unique()[-2])) & (dfFinal.ROTURA=="SIM")].head(10)

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT_1_Dias_Antes,EXPECTED_1_Dias_Antes,PRES_STOCK_1_Dias_Antes,STOCK,STOCK_1_Dias_Antes,...,Dias_Duracao_Linear_5_Dias_Antes,Desempenho_de_Vendas_60,Volatilidade_SELLOUT,Sensibilidade_Rotura,PS_Classificacao_60,RISCO,RISCO_ROTURA,Conclusao1,Conclusao2,Conclusao3
64975,2023-03-21,5601082049946,CAFÉ BELLISSIMO INTENSO 200GR,2,CNT AMADORA,0.0,0.0,35,0.0,8.0,...,29.560606,Vendas com desempenho insatisfatório,e com alta volatilidade de sellout.,Produto com muito baixa ou nula propensão par...,e com PS equilibrado.,3,O produto apresenta hoje muito elevada probab...,Vendas com desempenho insatisfatório e com alt...,Produto com muito baixa ou nula propensão par...,O produto apresenta hoje muito elevada probab...
64976,2023-03-22,5601082049946,CAFÉ BELLISSIMO INTENSO 200GR,2,CNT AMADORA,0.0,0.0,35,0.0,0.0,...,39.128676,Vendas com desempenho mediano,e com alta volatilidade de sellout.,Produto com baixa propensão para rotura,e com PS equilibrado.,3,O produto apresenta hoje média probabilidade ...,Vendas com desempenho mediano e com alta volat...,Produto com baixa propensão para rotura e com...,O produto apresenta hoje média probabilidade ...
64977,2023-03-23,5601082049946,CAFÉ BELLISSIMO INTENSO 200GR,2,CNT AMADORA,0.0,80.0,35,0.0,0.0,...,37.817511,Vendas com desempenho mediano,e com alta volatilidade de sellout.,Produto com moderada propensão para rotura,e com PS equilibrado.,3,O produto apresenta hoje baixa probabilidade ...,Vendas com desempenho mediano e com alta volat...,Produto com moderada propensão para rotura e ...,O produto apresenta hoje baixa probabilidade ...
65020,2023-05-05,5601082049946,CAFÉ BELLISSIMO INTENSO 200GR,2,CNT AMADORA,0.0,128.0,35,0.0,6.0,...,32.370249,Vendas com desempenho mediano,e com média volatilidade de sellout.,Produto com moderada propensão para rotura,e com PS equilibrado.,3,O produto apresenta hoje baixa probabilidade ...,Vendas com desempenho mediano e com média vola...,Produto com moderada propensão para rotura e ...,O produto apresenta hoje baixa probabilidade ...


- Passar para csv

In [369]:
dfMachineLearning = dfFinal[["DATA", "STORE", "ROTURA", "CICLOS", "Balance_Smart", 
                      "Percentagem_Dias_Stock_Borderline_60","Percentagem_Dias_Stock_Borderline_120", 
                      "Percentagem_Roturas_60", "Percentagem_Roturas_120", 
                      "Percentagem_Volatilidade_60", "Percentagem_Volatilidade_120", 
                      "Percentagem_Dias_Sem_Vendas_60","Percentagem_Dias_Sem_Vendas_120",
                      "Percentagem_Dias_Linear_Incompleto_60", "Percentagem_Dias_Linear_Incompleto_120",
                      "Percentagem_Balance_Raw_Count1_60", "Percentagem_Balance_Raw_Count1_120",
                      "Percentagem_Balance_Raw_Count2_60", "Percentagem_Balance_Raw_Count2_120"]]

# Clickhouse

In [370]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX')

In [371]:
#Estabelecer a base a ser lida

qualbase = 2

if qualbase == 1:
    baseCH = dfEscreverDia.copy()
    tabela = "BaseDeltaDia"

elif qualbase == 2:
    baseCH = dfEscreverMes.copy()
    tabela = "BaseDeltaMes"

elif qualbase == 3:
    baseCH = dfEscrever180.copy()
    tabela = "BaseDelta180"




In [372]:



###
baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar para datetime
new_columns = [unidecode(col) for col in baseCH.columns]           # Tirar acentos e afins
baseCH.columns = new_columns                                       # Aplicar alterações da linha anterior
###

# Tipos de dados
data = ["DATA"]
texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


# Só floats é que permitem missing values
for col_name in inteiros:
    baseCH[col_name] = baseCH[col_name].astype(float)
# Missing values em strings estragam tudo
baseCH[texto] = baseCH[texto].fillna("-")


def schema(lista, tipo):
    result_list = [f"{element} {tipo}" for element in lista]
    return result_list

data1 = schema(data, "Date")
texto1 = schema(texto, "String")
inteiros1 = schema(inteiros, "Float64")
floats1 = schema(floats, "Float64")
total = tuple(data1 + texto1 + inteiros1 + floats1)

schema = ', '.join([column.replace("'", "") for column in total])

# Split the input string by commas
parts = schema.split(', ')
# Process each part and wrap the first word in double quotes
output_parts = []
for part in parts:
    words = part.split()
    if words:
        first_word = words[0]
        remaining_words = ' '.join(words[1:])
        output_part = f'"{first_word}" {remaining_words}'
        output_parts.append(output_part)
# Join the modified parts back into a string
schema = ', '.join(output_parts)



# Eliminar tabela no CH
client.command(f'DROP TABLE IF EXISTS {tabela}')

# Criar tabela no CH
client.command(f'''
    CREATE TABLE IF NOT EXISTS {tabela} (
        {schema}
        ) ENGINE = MergeTree
        ORDER BY (DATA)
''')

client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())

In [373]:
colunas = dfEscrever180.columns.tolist()[0:18]
#colunas = ['DATA','EAN','DESC_ARTIGO','STORE',...]
#colunas